In [ ]:
import numpy as np
import torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn

## Load Dataset

In [ ]:
download_root = './data'

In [ ]:
# Normalize data with mean=0.1307, std=0.3081
mnist_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

In [ ]:
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

## Data Loader

In [ ]:
batch_size = 20

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler

valid_size = 0.3
num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))

np.random.shuffle(indices)
    
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=batch_size,
                          sampler=train_sampler)

valid_loader = DataLoader(dataset=valid_dataset, 
                         batch_size=batch_size,
                          sampler=valid_sampler)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         shuffle=True)

In [ ]:
num_train = len(train_loader) * batch_size
num_valid = len(valid_loader) * batch_size
num_test = len(test_loader) * batch_size
num_train, num_valid, num_test

## Define a model

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(5*5*64, 1024)
        self.fc2 = nn.Linear(1024, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 5*5*64)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
model = CNN()

In [ ]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print("we are using GPU!")
    model = model.cuda()

## Define Optimizer

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Define Criterion 

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
save_path = "./model"

In [ ]:
model.train()
best_loss = 9999.0

for epoch in range(20):
    avg_loss = 0
    model.train()
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        output = model(x)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 300 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx, len(train_loader),
                100. * batch_idx / len(train_loader), loss.item()))
            
    model.eval()
    
    valid_loss=0
    valid_correct=0
    
    for batch_idx, (x, target) in enumerate(valid_loader):
        bb+=1
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        output = model(x)
        valid_loss += criterion(output, target).item() # sum up batch loss
        
    valid_loss /= batch_idx
    print("[validation] valid_loss: {:.3f}".format(valid_loss))
    if valid_loss < best_loss:
        print("[validation] saving models {}".format(valid_loss))
        torch.save(model.state_dict(), save_path)
        best_loss = valid_loss

## Load Saved Models

In [ ]:
model.load_state_dict(torch.load(save_path))

## Test Models

In [ ]:
model.eval()

test_loss = 0
correct = 0
with torch.no_grad():
    for x, target in test_loader:
        if use_cuda:
            x, target = x.cuda(), target.cuda()
            output = model(x)
        test_loss += criterion(output, target).item() # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= num_test

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / num_test))